# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [2]:
import os
import sys

sys.path.append('..')
# Set your preferred name
USER = "Yuxuan"
# Set the data and output directories
DATA_ROOT = '/home/yxcheng'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Duchenne-Muscular-Dystrophy'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [3]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [4]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

['TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Lower_Grade_Glioma_(LGG)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Ocular_melanomas_(UVM)',
 'TCGA_Ovarian_Cancer_(OV)',
 'TCGA_Prostate_Cancer_(PRAD)',
 'CrawlData.ipynb',
 'TCGA_Kidney_Papillary_Cell_Carcinoma_(KIRP)',
 'TCGA_Lung_Cancer_(LUNG)',
 'TCGA_Endometrioid_Cancer_(UCEC)',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)',
 'TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Glioblastoma_(GBM)',
 'TCGA_Melanoma_(SKCM)',
 'TCGA_Kidney_Clear_Cell_Carcinoma_(KIRC)',
 'TCGA_Mesothelioma_(MESO)',
 'TCGA_Rectal_Cancer_(READ)',
 'TCGA_Head_and_Neck_Cancer_(HNSC)',
 'TCGA_Large_Bcell_Lymphoma_(DLBC)',
 'TCGA_Uterine_Carcinosarcoma_(UCS)',
 'TCGA_Colon_Cancer_(COAD)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Bladder_Cancer_(BLCA)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)',
 'TCGA_Cervical_Ca

If no match is found, jump directly to GEO in Part 2.2

In [27]:
trait_subdir = "TCGA_Lung_Cancer_(LUNG)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_LUNG_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_HiSeqV2_PANCAN.gz')

In [28]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [ ]:
clinical_data_cols[:10]

['ABSOLUTE_Ploidy',
 'ABSOLUTE_Purity',
 'AKT1',
 'ALK_translocation',
 'BRAF',
 'CBL',
 'CTNNB1',
 'Canonical_mut_in_KRAS_EGFR_ALK',
 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2',
 'EGFR']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['ABSOLUTE_Ploidy', 'ABSOLUTE_Purity', 'AKT1', 'ALK_translocation', 'BRAF', 'CBL', 'CTNNB1', 'Canonical_mut_in_KRAS_EGFR_ALK', 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2', 'EGFR', 'ERBB2', 'ERBB4', 'Estimated_allele_fraction_of_a_clonal_varnt_prsnt_t_1_cpy_pr_cll', 'Expression_Subtype', 'HRAS', 'KRAS', 'MAP2K1', 'MET', 'NRAS', 'PIK3CA', 'PTPN11', 'Pathology', 'Pathology_Updated', 'RET_translocation', 'ROS1_translocation', 'STK11', 'WGS_as_of_20120731_

In [ ]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'stopped_smoking_year', 'year_of_initial_pathologic_diagnosis', 'year_of_tobacco_smoking_onset']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [70.0, 67.0, 79.0, 68.0, 66.0],
 'days_to_birth': [-25752.0, -24532.0, -29068.0, -24868.0, -24411.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': [nan, nan, '121', nan, nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': ['0', '1523', nan, '607', '426'],
 'days_to_new_tumor_event_after_initial_treatment': [nan,
  nan,
  nan,
  334.0,
  183.0],
 'stopped_smoking_year': [nan, nan, nan, nan, 1987.0],
 'year_of_initial_pathologic_diagnosis': [2009.0,
  2007.0,
  2007.0,
  2009.0,
  2009.0],
 'year_of_tobacco_smoking_onset': [nan, nan, nan, nan, 1963.0]}

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Lung-Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-05-4244-01,1,70.0,1.0,-0.056592,2.128674,-0.531035,-0.185972,-0.190878,-0.11891,-0.659794,...,0.015022,-1.155117,0.528367,2.75061,0.428423,-0.639586,-0.195527,-0.324695,0.193167,-0.572812
TCGA-05-4249-01,1,67.0,1.0,-0.569692,-0.318126,-0.531035,0.237428,0.011822,-0.00611,1.018406,...,-0.031778,-0.393117,1.632467,2.33551,-0.145577,-0.997486,-0.276027,-1.219395,1.850967,-0.471212
TCGA-05-4250-01,1,79.0,0.0,-0.899992,-0.918126,-0.531035,-0.621472,-0.454178,0.58719,-0.198894,...,-0.748878,-1.173917,1.966367,-1.28139,-0.374877,0.959514,0.714373,0.112305,-0.532833,-0.003812
TCGA-05-4382-01,1,68.0,1.0,0.667508,3.895974,-0.221535,-0.367272,0.131822,0.97629,-0.494994,...,1.013722,-1.587117,0.359267,2.59051,0.806623,0.812114,0.367373,-0.494595,-1.062233,0.108588
TCGA-05-4384-01,1,66.0,1.0,-0.316892,2.380374,0.156865,0.232828,0.054822,0.27059,0.128606,...,-0.060978,-0.435417,1.111467,-1.28139,-0.128477,0.110214,1.469373,0.194605,1.404667,-0.290712


In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1100 samples.


In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Lung-Cancer', the least common label is '0' with 110 occurrences. This represents 10.00% of the dataset.
The distribution of the feature 'Lung-Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 67.0
  75%: 73.0
Min: 38.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 443 occurrences. This represents 40.27% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Yuxuan/Lung-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [5]:
dataset = 'GEO'
trait_subdir = "Duchenne-Muscular-Dystrophy"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE85017',
 'GSE60909',
 'GSE60910',
 'GSE109178',
 'GSE79263',
 'GSE120624',
 'GSE3307',
 'GSE465',
 'GSE1007',
 'GSE13608',
 'GSE6011',
 'GSE48828']

Repeat the below steps for all the accession numbers

In [60]:
cohort = accession_num = "GSE48828"
cohort_dir = os.path.join(trait_path, accession_num)
#print(cohort_dir)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/home/yxcheng/GEO/Duchenne-Muscular-Dystrophy/GSE48828/GSE48828_family.soft.gz',
 '/home/yxcheng/GEO/Duchenne-Muscular-Dystrophy/GSE48828/GSE48828-GPL5188_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [61]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Expression and Splicing Analysis of Myotonic Dystrophy and Other Dystrophic Muscle"
!Series_summary	"The prevailing patho-mechanistic paradigm for myotonic dystrophy (DM) is that the aberrant presence of embryonic isoforms is responsible for many, if not most, aspects of the pleiotropic disease phenotype.  In order to identify such aberrantly expressed isoforms in skeletal muscle of DM type 1 (DM1) and type 2 (DM2) patients, we utilized the Affymetrix exon array to characterize the largest collection of DM samples analyzed to date, and included non-DM dystrophic muscle samples (NMD) as disease controls."
!Series_overall_design	"For the exon array profiling on the Human Exon 1.0 ST array (Affymetrix Santa Clara, CA) we used a panel of 28 skeletal muscle biopsies from DM1 (n=8), DM2 (n=10), Becker muscular dystrophy, BMD, (n=3), Duchenne muscular dystrophy, DMD (n=1), Tibial muscular dystrophy, TMD, (n=2) and normal skeletal muscle (n=4).  Normal control RNAs were purchase

In [62]:
clinical_data.head()

,!Sample_geo_accession,GSM1185341,GSM1185342,GSM1185343,GSM1185344,GSM1185345,GSM1185346,GSM1185347,GSM1185348,GSM1185349,...,GSM1185359,GSM1185360,GSM1185361,GSM1185362,GSM1185363,GSM1185364,GSM1185365,GSM1185366,GSM1185367,GSM1185368
0,!Sample_characteristics_ch1,diagnosis: Myotonic Dystrophy Type 1,diagnosis: Myotonic Dystrophy Type 1,diagnosis: Myotonic Dystrophy Type 1,diagnosis: Myotonic Dystrophy Type 1,diagnosis: Myotonic Dystrophy Type 1,diagnosis: Myotonic Dystrophy Type 1,diagnosis: Myotonic Dystrophy Type 1,diagnosis: Myotonic Dystrophy Type 1,diagnosis: Myotonic Dystrophy Type 2,...,diagnosis: Becker Muscular Dystrophy,diagnosis: Becker Muscular Dystrophy,diagnosis: Becker Muscular Dystrophy,diagnosis: Duchenne Muscular Dystrophy,diagnosis: Tibial muscular Dystophy,diagnosis: Tibial muscular Dystophy,diagnosis: Normal,diagnosis: Normal,diagnosis: Normal,diagnosis: Normal
1,!Sample_characteristics_ch1,gender: F,gender: F,gender: F,gender: F,gender: M,gender: M,gender: F,gender: Not available,gender: M,...,gender: M,gender: M,gender: M,gender: M,gender: M,gender: F,gender: F,gender: M,gender: M,gender: M
2,!Sample_characteristics_ch1,age (yrs): Not available,age (yrs): Not available,age (yrs): 54,age (yrs): 29,age (yrs): 25,age (yrs): 21,age (yrs): 55,age (yrs): na,age (yrs): 39,...,age (yrs): 50,age (yrs): 45,age (yrs): 26,age (yrs): 20,age (yrs): 58,age (yrs): 88,age (yrs): 61,age (yrs): 43,age (yrs): 85,age (yrs): 43


In [64]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['diagnosis: Myotonic Dystrophy Type 1',
  'diagnosis: Myotonic Dystrophy Type 2',
  'diagnosis: Becker Muscular Dystrophy',
  'diagnosis: Duchenne Muscular Dystrophy',
  'diagnosis: Tibial muscular Dystophy',
  'diagnosis: Normal'],
 1: ['gender: F', 'gender: M', 'gender: Not available'],
 2: ['age (yrs): Not available',
  'age (yrs): 54',
  'age (yrs): 29',
  'age (yrs): 25',
  'age (yrs): 21',
  'age (yrs): 55',
  'age (yrs): na',
  'age (yrs): 39',
  'age (yrs): 58',
  'age (yrs): 50',
  'age (yrs): 51',
  'age (yrs): 43',
  'age (yrs): 37',
  'age (yrs): 65',
  'age (yrs): 45',
  'age (yrs): 26',
  'age (yrs): 20',
  'age (yrs): 88',
  'age (yrs): 61',
  'age (yrs): 85']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [65]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Duchenne-Muscular-Dystrophy\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Duchenne-Muscular-Dystrophy\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this tra

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [66]:
is_gene_availabe = True
trait_row = 1
age_row = 2
gender_row = 3

trait_type = 'binary'

In [67]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [72]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'diagnosis: Duchenne Muscular Dystrophy':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present

def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age (yrs): '):
        try:
            return int(age_string.split('age (yrs): ')[1].rstrip(''))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format

def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'gender: F':
        return 0
    elif gender_string == 'gender: M':
        return 1
    else:
        return None  # In case of unknown gender

# Example usage
example_tissue = 'diagnosis: Duchenne Muscular Dystrophy'
example_age = 'age: 23y'
example_gender = 'gender: Male'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

(1, None, None)

In [73]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

/home/yxcheng/AI4Science_gold_standard/stage1/YIjiang/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
/home/yxcheng/AI4Science_gold_standard/stage1/YIjiang/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)


ValueError: Length mismatch: Expected axis has 0 elements, new values have 1 elements

### Genetic data preprocessing and final filtering

In [40]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM343029,GSM343030,GSM343031,GSM343032,GSM343033,GSM343034,GSM343035,GSM343036,GSM343037,GSM343038,...,GSM343087,GSM343088,GSM343089,GSM343090,GSM343091,GSM343092,GSM343093,GSM343094,GSM343095,GSM343096
ID,,,,,,,,,,,,,,,,,,,,,
1007_s_at,483.00,446.71,471.33,554.88,265.14,537.91,305.40,400.95,387.56,434.66,...,478.38,535.23,564.40,497.93,564.13,495.51,485.00,493.99,606.13,426.88
1053_at,178.24,241.94,221.67,157.66,323.58,194.50,137.92,188.71,150.00,170.15,...,253.80,148.84,168.53,165.99,158.71,164.88,209.96,204.70,220.92,167.70
117_at,134.76,144.44,139.10,130.97,140.05,117.22,123.53,132.29,140.60,148.24,...,132.15,104.44,122.84,134.33,119.06,150.41,122.16,126.74,123.71,159.82
121_at,248.38,268.66,284.44,221.12,241.72,198.43,215.95,249.54,275.08,252.60,...,227.40,190.94,202.90,218.38,200.36,245.24,263.00,223.32,225.34,273.48
1255_g_at,54.59,72.05,63.95,61.49,71.16,58.16,67.13,58.27,63.75,59.39,...,58.66,55.38,60.09,66.52,62.69,61.23,66.07,64.08,57.31,99.45


In [41]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['1007_s_at',
 '1053_at',
 '117_at',
 '121_at',
 '1255_g_at',
 '1294_at',
 '1316_at',
 '1320_at',
 '1405_i_at',
 '1431_at',
 '1438_at',
 '1487_at',
 '1494_f_at',
 '1552256_a_at',
 '1552257_a_at',
 '1552258_at',
 '1552261_at',
 '1552263_at',
 '1552264_a_at',
 '1552266_at']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [42]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at', '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at', '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at', '1552263_at', '1552264_a_at', '1552266_at']\n"

If not required, jump directly to the gene normalization step

In [43]:
requires_gene_mapping = True

In [44]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEFINITION=HSPAX8A H.sapiens

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [45]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Da

In [46]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'Gene Symbol'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [47]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

155 input query terms found dup hits:	[('ABCC6P1', 2), ('ABCC6P2', 3), ('ABCD1P2', 2), ('ACTBP9', 2), ('ACTR3BP5', 3), ('ADAM1A', 2), ('AG
409 input query terms found no hit:	['AC078883.4', 'AF279782', 'AK023507', 'AK055364', 'AK055967', 'AK056973', 'AK094915', 'AK096803', '
129 input query terms found dup hits:	[('LONP2', 2), ('LRRC37A16P', 2), ('LRRC37A2', 2), ('LRRC37A4P', 3), ('LRRC37A5P', 2), ('LRRC37A7P',
206 input query terms found no hit:	['LOC285389', 'LOC285835', 'LOC388572', 'LOC388692', 'LOC399491', 'LOC400927', 'LOC401317', 'LOC4013
106 input query terms found dup hits:	[('SNORD29', 2), ('SNORD33', 2), ('SNORD36B', 2), ('SNORD38B', 2), ('SNORD3B-1', 2), ('SNORD3B-2', 2
186 input query terms found no hit:	['SOGA3', 'SOX5P', 'SPHAR', 'SSU72P8', 'TCEB1P28', 'TCEB1P3', 'TCEB3C', 'TCEB3CL', 'TCEB3CL2', 'TCR-
58 input query terms found dup hits:	[('AFG3L1P', 2), ('AGAP11', 4), ('AGAP6', 2), ('AGO2', 2), ('AHCTF1', 2), ('AK6', 2), ('AKIP1', 2), 
156 input query terms found no hit

In [48]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [49]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 49 samples.


In [50]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Duchenne-Muscular-Dystrophy', the least common label is '1.0' with 3 occurrences. This represents 6.12% of the dataset.
The distribution of the feature 'Duchenne-Muscular-Dystrophy' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 39.0
  50% (Median): 50.0
  75%: 58.0
Min: 5.0
Max: 88.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 20 occurrences. This represents 40.82% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



True

In [51]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

A new JSON file was created at: ../../output1/Yuxuan/Duchenne-Muscular-Dystrophy/cohort_info.json


In [52]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [53]:
from utils import *

In [54]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

In [55]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note


In [74]:
ck=100
listq=[]
for df in pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'),chunksize=ck):
    listq.append(df)
merged_data = pd.concat(listq)
#merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [ ]:
merged_data.head()

,Psoriasis,Age,Gender,-0.001001579,-0.010425045,-0.045320336,-0.046967808,-0.069891929,-0.081080626,-0.093186199,...,9987,9988,9989,999,9990,9991,9992,9993,9994,9997
0,0.0,56.0,1.0,3.293947,3.344254,3.049427,3.489313,3.713489,3.182711,2.885457,...,11.631593,9.914236,5.934949,4.756848,10.265351,10.109506,4.033606,11.065690,8.733339,9.681242
1,0.0,20.0,0.0,2.295976,3.490590,3.402046,3.960615,2.295976,3.209781,3.380016,...,11.685883,9.888024,5.981049,3.573693,10.808423,10.135307,3.854406,10.578108,8.207179,9.830063
2,0.0,51.0,0.0,2.878075,3.642838,3.281445,3.630912,2.878075,3.658581,3.466199,...,11.823756,9.977897,5.965869,4.439257,10.669074,10.243314,4.174246,10.824677,8.554421,10.367084
3,0.0,37.0,1.0,3.567006,3.393262,3.477423,3.000214,3.190889,3.070480,2.543753,...,11.871873,9.948733,6.094711,4.479720,10.832092,10.253960,3.735004,10.505996,8.786537,9.695719
4,0.0,61.0,1.0,4.034010,3.607132,2.822304,3.712049,3.126172,3.017512,2.949723,...,11.931741,10.017533,6.085654,3.807143,10.252109,10.396369,4.112947,10.797622,8.826644,8.821104


In [ ]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [ ]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [ ]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [ ]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [ ]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

The cross-validation accuracy is 56.84% ± 4.59%


In [ ]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

In [ ]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

### 5. Discussion and report

In [ ]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0025
Found 547 significant genes associated with the trait 'Psoriasis' conditional on the factor 'Age', with corrected p-value < 0.05:
   Variable  Coefficient  corrected_p_value
4.430015695     0.087750           0.001231
4.478193647     0.087750           0.001231
4.478966158     0.087750           0.001231
 4.48687579     0.087750           0.001231
4.491851159     0.087750           0.001231
4.492453853     0.087750           0.001231
4.494270678     0.087750           0.001231
4.504245863     0.087750           0.001231
4.505190735     0.087750           0.001231
4.505235806     0.087750           0.001231
4.506891502     0.087750           0.001231
4.510669146     0.087750           0.001231
4.476613942     0.087750           0.001231
 4.51198966     0.087750           0.001231
4.513528153     0.087750           0.001231
4.516930407     0.087750           0.001231
4.806825082     0.087750           0.001

/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/utils.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values


In [ ]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)